In [5]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import random
import pickle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from bs4 import BeautifulSoup
import re
from scipy.stats import norm
from bayes_opt import BayesianOptimization
import graphviz
import warnings
warnings.simplefilter('ignore')
def make_Xyqodds():
    result = [[[],[],[],[]],[[],[],[],[]]]
    with open("keiba_2019",mode="r") as f:
        for line in f.readlines():
          
            year = int(line[0:4])
            if year < 2020:
                num = 0
            else:
                num = 1
        
            
            line_list = line.split(",")
            race_info,horses_info,odds_info = line_list[0],line_list[1:-1],line_list[-1]
            race_list = race_info.split(";")
            for horse_info in horses_info:
                horse_list = horse_info.split(";")
                label = int(18/int(horse_list[0]))
                data = race_list + horse_list[1:] + [race_list[0][4:6]] + [race_list[0][-2:]]
                for i in [2,5,8,10,11,12,18]:
                    try:
                        data[i] = float(data[i])
                    except:
                        data[i] = 0
                result[num][0].append(data)
                result[num][1].append(label)
            
            result[num][2].append(len(horses_info))
            result[num][3].append(odds_info)
    result = np.array(result)

    return result
Xyqodds = make_Xyqodds()
def train_sim(max_depth,num_leaves,max_position):
    
    boosting_type = "gbdt"
    result = Xyqodds
  
    X_train,X_test = result[0][0],result[1][0]
    y_train,y_test = result[0][1],result[1][1]
    q_train,q_test = result[0][2],result[1][2]
    odds_train,odds_test = result[0][3],result[1][3]
    category = [0,1,3,4,6,7,9,13,14,15,16,17,19,20]
    number =[2,5,8,10,11,12,18]
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    for i in category:
        X_train[i] = X_train[i].astype('category')
        X_test[i] = X_test[i].astype('category')
    #for i in number:
        #X_train[i] = X_train[i].astype('float')
        #X_test[i] = X_test[i].astype('float')

    
    X_train.drop(columns=10,inplace=True) 
    X_test.drop(columns=10,inplace=True) 

    train = lgb.Dataset(X_train, y_train,group=q_train)
    test = lgb.Dataset(X_test, y_test, group=q_test)

    params = { 
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_eval_at': 3,
        'max_depth': int(max_depth),
        'max_position':int(max_position), 
        'num_leaves': int(num_leaves),
        'min_data_in_leaf':1,
        'verbose': -1,
        'categorical_feature':category,
        'boosting_type': boosting_type
    }
    #print(type(X_train.loc[0,2]))
    model = lgb.train(
        params,
        train,
        num_boost_round=150,
        valid_sets=[train,test],
        valid_names=['train','test'],
        early_stopping_rounds=20,
        verbose_eval=False
    )

    from_ = 0
    odds_all = odds_test
   
  
    gain = [[],[],[],[],[],[],[]]
    for i,odds in zip(q_test,odds_all):
        pred = model.predict(X_test[from_:from_+i],num_iteration=model.best_iteration)
        pred = pd.Series(pred).sort_values(ascending=False)          
        pred.index = pred.index + 1
        pred_rank = list(map(str,pred.index.values))[0:3]

        from_ += i
        odds_list = re.findall(r'([^\|]*)\|([^\|]*)\|([^;]*)円;?',odds)

        result = []
        for i,j in zip([1,2],[" ","円"]):
            comb_list =  [x[i] for x in odds_list if x[0] != "枠連"]
            comb_list[0] = [comb_list[0:1]]
            comb_list[1] = [[a] for a in comb_list[1].split(j)]
            if len(comb_list[1]) == 5:
                comb_list[1] = comb_list[1][::2]
                j = "   "
            comb_list[2] = [set(comb_list[2].split(j))]
            comb_list[3] = comb_list[3].split(j)
            comb_list[3] = [set(comb_list[3][0:(3-i)]),set(comb_list[3][3-i:6-2*i]),set(comb_list[3][6-2*i:9-3*i])]
            comb_list[4] = [comb_list[4].split(j)]
            comb_list[5] = [set(comb_list[5].split(j))]
            comb_list[6] = [comb_list[6].split(j)]
            result.append(comb_list)

        baken_list = [0,0,0,0,0,0,0]
        baken_list[0] = [[a] for a in pred_rank[0:1]]
        baken_list[1] = [[a] for a in pred_rank[0:3]]
        baken_list[2] = [set(pred_rank[0:2])]
        baken_list[3] = [set(pred_rank[0:2]),set(pred_rank[1:3]),set(pred_rank[::2])]
        baken_list[4] = [pred_rank[0:2]]
        baken_list[5] = [set(pred_rank)]
        baken_list[6] = [pred_rank]
        
        for i in range(len(result[0])):
            for j in range(len(result[0][i])):
                if result[0][i][j] in baken_list[i]:
                    gain[i].append(float(list(result[1][i][j])[0]))
                else:
                    gain[i].append(0)
                



    gain = np.array(gain)
    objective = []
    for i in range(7):
        p = norm.sf(x=100,loc=np.average(gain[i]),scale=np.std(gain[i],ddof=1)/np.sqrt(len(gain[i])))
        objective.append(p)
    return max(objective)

rf_cv_bo = BayesianOptimization(
    train_sim, # 上の関数を与える
    {'max_depth': (1,10),
     'num_leaves': (2,1024),
     'max_position':(1,10)
    },
    verbose=2, # 0だと学習過程を表示をしない、デフォルトの2なら全て表示する、1は最高値が更新されたら表示
    random_state=0
)
                
rf_cv_bo.maximize(init_points=10, n_iter=100, acq='ei')

# 実行結果の内、最も精度の良かった評価値とパラメータの値をprint
print(rf_cv_bo.max['target'])

print('max_depth:', rf_cv_bo.max['params']['max_depth'])
print('num_leaves:', rf_cv_bo.max['params']['num_leaves'])
print('boosting_type:', rf_cv_bo.max['params']['boosting_type'])

|   iter    |  target   | max_depth | max_po... | num_le... |
-------------------------------------------------------------
|  1        |  0.2407   |  5.939    |  7.437    |  618.0    |
|  2        |  0.2407   |  5.904    |  4.813    |  662.1    |
|  3        |  0.403    |  4.938    |  9.026    |  986.9    |
|  4        |  0.403    |  4.451    |  8.126    |  542.5    |
|  5        |  0.2033   |  6.112    |  9.33     |  74.6     |
|  6        |  0.835    |  1.784    |  1.182    |  852.9    |
|  7        |  0.1125   |  8.003    |  8.83     |  1.002e+0 |
|  8        |  0.08951  |  8.192    |  5.153    |  799.7    |
|  9        |  0.2924   |  2.064    |  6.759    |  148.5    |
|  10       |  0.5276   |  9.502    |  5.697    |  425.8    |
|  11       |  0.835    |  1.0      |  1.0      |  908.3    |
|  12       |  0.835    |  1.0      |  1.0      |  297.2    |
|  13       |  0.5668   |  10.0     |  10.0     |  342.8    |
|  14       |  0.4993   |  10.0     |  10.0     |  250.3    |
|  15   

|  81       |  0.835    |  1.0      |  1.0      |  210.2    |
|  82       |  0.835    |  1.0      |  10.0     |  230.7    |
|  83       |  0.1358   |  10.0     |  1.0      |  402.0    |
|  84       |  0.835    |  1.0      |  10.0     |  432.6    |
|  85       |  0.835    |  1.0      |  1.0      |  430.9    |
|  86       |  0.2665   |  10.0     |  1.0      |  586.2    |
|  87       |  0.835    |  1.0      |  10.0     |  581.6    |
|  88       |  0.835    |  1.0      |  10.0     |  567.3    |
|  89       |  0.835    |  1.0      |  1.0      |  336.1    |
|  90       |  0.835    |  1.0      |  10.0     |  950.2    |
|  91       |  0.835    |  1.0      |  10.0     |  421.8    |
|  92       |  0.835    |  1.0      |  1.0      |  272.2    |
|  93       |  0.835    |  1.0      |  10.0     |  896.5    |
|  94       |  0.04994  |  10.0     |  1.0      |  362.0    |
|  95       |  0.835    |  1.0      |  10.0     |  346.6    |
|  96       |  0.6735   |  7.528    |  1.847    |  380.7    |
|  97   

KeyError: 'boosting_type'

In [61]:
#モデルの構築(ベイズ最適化で最大利益率のもの)
import numpy as np
from sklearn import preprocessing 
import pandas as pd

l = [["a","b"],[4,5]]
l = pd.DataFrame(l)
l[0].astype("category")
float("14")
boosting_type = 0.4
boosting_type = "dart" if boosting_type > 0.5 else "gbdt"
boosting_type

'gbdt'

In [63]:
print(get)

NameError: name 'get' is not defined

In [4]:
from scipy.stats import norm
print(norm.sf(x=100,loc=156,scale=60))

0.8246760551477705
